## Tests for Ishigami function
Exact permutation. Analytical Shapley values obtained from Plischke, Rabitti, and Borgonovo (2020), p. 8. Inputs are independent.

In [1]:
import numpy as np
import pandas as pd
import chaospy as cp
from numpy.testing import assert_array_almost_equal as aaae

from econsa_shapley import get_shapley
from econsa_shapley import _r_condmvn
from econsa_sampling import cond_mvn

In [50]:
# Function inputs.
n_inputs = 8
perms = np.random.permutation(n_inputs)
j = 4
subset_j = perms[:j]
subsetj_conditional = perms[j:]
mean = np.zeros(n_inputs)
cov = np.identity(n_inputs)
cov

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [17]:
# What does x_cond from the test cases spit out?
def x_cond(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)  # Get an array from cov.
        cov_int = cov_int.take(subset_j, axis=1)  # Take only columns/rows (depending on axis) that correspond to the indices
        #provided by subset_j.
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)
        return distribution.sample(n)
    else:
        return _r_condmvn(
            n,
            mean=mean,
            cov=cov,
            dependent_ind=subset_j,
            given_ind=subsetj_conditional,
            x_given=xjc,
            )
    
n_outer = 10
xjc_sampled = np.array(x_cond(n_outer, subset_j=subset_j, subsetj_conditional=None, xjc=None)).T
xjc_sampled

array([[ 0.50579794, -1.65869931, -1.87490569, -0.36356635],
       [-0.36251082,  0.63174304, -1.52049426,  0.36457521],
       [-0.32817038,  1.4366017 , -0.33510075, -0.25335804],
       [-0.16649476, -0.56722766, -0.23106494, -0.60372966],
       [-1.13534391, -0.29001338, -0.96210995, -1.27849867],
       [ 1.87641278, -0.0494357 , -0.71026049,  0.33427107],
       [ 1.04464395,  0.69712425,  1.36093197,  0.43008447],
       [ 0.87223662,  0.91828618, -1.34210354,  1.21047442],
       [-1.14699892,  0.35333631, -0.52183938,  1.08274256],
       [ 0.3286889 , -1.07269878, -0.83712016,  0.70489209]])

In [35]:
n_inner = 4
n_inputs = 8
mean = np.zeros(n_inputs)
cov = np.identity(n_inputs)

for j in range(1, n_inputs):
    
    perms = np.random.permutation(n_inputs)
    j = 4
    subset_j = perms[:j]
    subsetj_conditional = perms[j:]
    for length in range(1, n_outer):
        
        xjc = xjc_sampled[length, ]
        
        sample_inputs = np.array(x_cond(n_inner, subset_j, subsetj_conditional, xjc.flat)).T.reshape(n_inner, -1,)

In [69]:
cov = np.identity(n_inputs)
cov_int = np.array(cov)
cov_int

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [70]:
subset_j

array([0, 1, 2, 6])

In [71]:
cov_int = cov_int.take(subset_j, axis=1)
cov_int

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [72]:
cov_int = cov_int[subset_j]
cov_int

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [73]:
distribution = cp.MvNormal(mean[subset_j], cov_int)

In [74]:
distribution.sample(3)

array([[ 0.06172266,  0.48953322, -0.72340657],
       [-0.56113574,  1.70222466, -0.03065108],
       [ 1.11076468,  0.36046205, -1.37229961],
       [ 2.3194623 ,  0.64655772, -1.22032668]])

In [7]:
n_inner = 4
test_ = x_cond(n_inner, subset_j, subsetj_conditional, xjc.flat)
#sample_inputs = np.array(x_cond(n_inner, subset_j, subsetj_conditional, xjc.flat)).T.reshape(n_inner, -1, )
#sample_inputs

ValueError: lengths of given_value and given_ind must be the same

In [13]:
    cond_mean, cond_var = cond_mvn(
        mean,
        cov,
        dependent_ind=dependent_ind,
        given_ind=given_ind,
        given_value=x_given,
    )
    distribution = cp.MvNormal(cond_mean, cond_var)

    return distribution.sample(n)

NameError: name 'dependent_ind' is not defined

In [ ]:
def test_get_shapley_ishigami():
    def ishigami_function(x):
        return math.sin(x[:, 1]) * (1 + 0.1 * (x[:, 3] ** 4)) + 7 * (math.sin(x[:, 2]) ** 2)

    def x_all(n):
        distribution = cp.Uniform(lower, upper)
        return distribution.sample(n)

    def x_cond(n, subset_j, subsetj_conditional, xjc):
        
        # Wrong, since we need samples for the variables corresponding to the indices in sj(c)!
        if subsetj_conditional is None:
            
            distribution = cp.Uniform(lower, upper)
        return distribution.sample(n)
            return # ndarray with dim n_sim x no. of elements in subset_j, exactly for those input variables.
        else:
            return # ndarray  with dim n_sim x no. of elements in subsetj_conditional.

    np.random.seed(123)
    n_inputs = 3
    mean = np.zeros(3)
    # Lower and upper bound of the uniform distribution.
    lower = -math.pi
    upper = math.pi
    variance = (1/12) * ((upper - lower) ** 2)
    cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
    method = "exact"
    n_perms = None
    n_output = 10 ** 4
    n_outer = 10 ** 3
    n_inner = 10 ** 2

    col = ["X" + str(i) for i in np.arange(n_inputs) + 1]
    names = ["Shapley effects", "std. errors", "CI_min", "CI_max"]

    expected = pd.DataFrame(
        data=[
            [0.101309, 0.418989, 0.479701],
            [0.00241549, 0.16297, 0.163071],
            [0.096575, 0.0995681, 0.160083],
            [0.106044, 0.73841, 0.79932],
        ],
        index=names,
        columns=col,
    ).T

    calculated = get_shapley(
        method,
        gaussian_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )

    aaae(calculated, expected)

In [21]:
# Example.
n_inputs = 7
n_sim = 10
perms = np.random.permutation(n_inputs)
j = 4
subset_j = perms[:j]
subsetj_conditional = perms[j:]
lower = 0
upper = 1
out = np.zeros((n_sim, n_inputs))
for i in subset_j:
    out[:, i] = cp.Uniform(lower, upper).sample(n_sim).T
out

array([[0.38377533, 0.        , 0.27999478, 0.        , 0.        ,
        0.47232937, 0.55148138],
       [0.16606031, 0.        , 0.49768564, 0.        , 0.        ,
        0.43442878, 0.46587021],
       [0.85391958, 0.        , 0.50341559, 0.        , 0.        ,
        0.36058198, 0.2781392 ],
       [0.53300291, 0.        , 0.50630399, 0.        , 0.        ,
        0.52169385, 0.53213383],
       [0.91287332, 0.        , 0.24418302, 0.        , 0.        ,
        0.0477079 , 0.12106805],
       [0.70604899, 0.        , 0.68216086, 0.        , 0.        ,
        0.84896816, 0.66360917],
       [0.57615931, 0.        , 0.23240094, 0.        , 0.        ,
        0.59900273, 0.65371224],
       [0.99222253, 0.        , 0.48436874, 0.        , 0.        ,
        0.73214416, 0.43902138],
       [0.02985574, 0.        , 0.72488381, 0.        , 0.        ,
        0.63781945, 0.09538967],
       [0.00731334, 0.        , 0.94950301, 0.        , 0.        ,
        0.25366271, 0.1

In [19]:
def x_cond(n, subset_j, subsetj_conditional, xjc):
        distribution = cp.Uniform(lower, upper)
        return distribution.sample(n)

In [ ]:
def x_cond(n, subset_j, subsetj_conditional, xjc):
        
        # Wrong, since we need samples for the variables corresponding to the indices in sj(c)!
        if subsetj_conditional is None:
            
            distribution = cp.Uniform(lower, upper)
        return distribution.sample(n)
            return # ndarray with dim n_sim x no. of elements in subset_j, exactly for those input variables.
        else:
            return # ndarray  with dim n_sim x no. of elements in subsetj_conditional.

In [ ]:
# Call the function.
x_cond(
    n=n_sim,
    subset_j=subset_j,
    subsetj_conditional=subsetj_conditional,
    
      )